In [1]:
from automol.pipeline import Pipeline

In [2]:
config_yaml = 'qm9_dataset_test_config.yaml'
pipeline = Pipeline(config_yaml)
pipeline.print_spec()

amount: 10
dataset_class: QM9
dataset_location: data/dsgdb9nsd.xyz
labels:
- homo
models_to_exclude:
- GaussianProcessRegressor
- MLPRegressor
- LinearRegression
- GradientBoostingRegressor
problem: regression



In [3]:
pipeline.spec['labels']

['homo']

In [4]:
pipeline.data_set.data['homo']

0   -0.2296
1   -0.2376
2   -0.2444
3   -0.2300
4   -0.2319
5   -0.2575
6   -0.1833
7   -0.1994
8   -0.2438
9   -0.2524
Name: homo, dtype: float64

In [5]:
pipeline.train()

     homo
0 -0.2296
1 -0.2376
2 -0.2444
3 -0.2300
4 -0.2319
5 -0.2575
6 -0.1833
7 -0.1994
8 -0.2438
9 -0.2524
-------------
     homo
0 -0.2296
1 -0.2376
2 -0.2444
3 -0.2300
4 -0.2319
5 -0.2575
6 -0.1833
-------------
     homo
7 -0.1994
8 -0.2438
9 -0.2524
<class 'pandas.core.frame.DataFrame'>
-------------
<class 'pandas.core.frame.DataFrame'>
-------------
<class 'pandas.core.frame.DataFrame'>


TypeError: predict() missing 1 required positional argument: 'x'

In [ ]:
pipeline.models[0].y.shape